In [18]:
from pathlib import Path
import math
from dataclasses import dataclass
from ipywidgets import interact, fixed
from solarmed_optimization.visualization import update_cycle, generate_optim_cycle_viz

%load_ext autoreload
%autoreload 2

# Paths definition
output_path: Path= Path("./results/optim_cycle_viz")
output_path.mkdir(exist_ok=True, parents=True)

# Time params definition
sample_time_mod: int = 600 # seconds
optim_window_size: int = 8 * 3600 # 12 hours
sample_time_opt: int = 1 * 3600 # 1 hour
episode_duration = 24*3600 #- optim_window_size

# Samples transformation
step_samples_opt: int = sample_time_opt // sample_time_mod
optim_window_samples: int = optim_window_size // sample_time_mod
episode_samples = episode_duration // sample_time_mod
n_evals_mod: int = optim_window_size  // sample_time_mod
default_n_dec_var_updates: int = optim_window_size // sample_time_opt

@dataclass
class DecisionVariablesUpdates:
    sf_active: int = default_n_dec_var_updates
    ts_active: int = default_n_dec_var_updates
    med_active: int = default_n_dec_var_updates
    med_vac_state: int = default_n_dec_var_updates
    qsf: int = default_n_dec_var_updates
    qts_src: int = default_n_dec_var_updates
    qmed_s: int = default_n_dec_var_updates
    qmed_f: int = default_n_dec_var_updates
    Tmed_s_in: int = default_n_dec_var_updates
    Tmed_c_out: int = default_n_dec_var_updates
    
    def __post_init__(self):
        # Validate that SfTs FSM related decision varaiables have the same number of updates
        assert self.sf_active == self.ts_active, "Solar field and thermal storage logical variables should have the same number of updates"
        
        # Validate that MED FSM related decision variables have the same number of updates
        assert self.med_active == self.med_vac_state, "MED logical variables should have the same number of updates"
        
        # TODO: Would be good to validate that the number of updates is within:
        # 1 <= n_uptes <= optim_window_size / sample_time_mod (=n_evals_mod)

dec_var_updates: DecisionVariablesUpdates = DecisionVariablesUpdates(
    qsf=20
)

# # Simulation
# idx_start_mod = 0
# for opt_step_idx in range(0, episode_samples//step_samples_opt):
#     idx_start_mod += step_samples_opt
#     optim_window_span: tuple[int, int] = (idx_start_mod, idx_start_mod + optim_window_size)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:

fig = generate_optim_cycle_viz(
    start = 0, 
    episode_span = episode_samples,
    optim_window_span = optim_window_samples,
    mod_eval_span = n_evals_mod,
    dec_var_updates=dec_var_updates,
    use_times=False
)

@interact(opt_step_idx=(0, (episode_samples-optim_window_samples)//step_samples_opt, 1), fig=fixed(fig))
def callback(opt_step_idx=10, fig=fig):
    update_cycle(fig=fig, opt_step_idx=opt_step_idx, step_size=step_samples_opt)

fig


interactive(children=(IntSlider(value=5, description='opt_step_idx', max=16, min=5), Output()), _dom_classes=(…

FigureWidget({
    'data': [{'marker': {'symbol': 'circle'},
              'mode': 'markers',
              'name': 'Model evaluations',
              'type': 'scatter',
              'uid': '4be50116-4dc3-4282-8631-62e598e95078',
              'x': array([30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47,
                          48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
                          66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]),
              'y': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])},
             {'marker': {'color': 'rgba(153, 170, 187, 1)', 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'Past model evaluations',
              'showlegend': False,
              'type': 'scatter',
              'uid': 'd62bdef9-8792-4f7c-83bb-3641d88a34c7',
       

In [21]:
fig = generate_optim_cycle_viz(
    start = 20000, 
    episode_span = episode_duration,
    optim_window_span = optim_window_size,
    mod_eval_span = sample_time_mod,
    dec_var_updates=dec_var_updates,
    use_times=True,
)

fig.update_layout(width=900)

# @interact(opt_step_idx=(0, (episode_samples-optim_window_samples)//step_samples_opt, 1), fig=fixed(fig))
# def callback(opt_step_idx=0, fig=fig):
#     update_cycle(fig=fig, opt_step_idx=opt_step_idx, step_size=step_samples_opt)


fig


FigureWidget({
    'data': [{'marker': {'symbol': 'circle'},
              'mode': 'markers',
              'name': 'Model evaluations',
              'type': 'scatter',
              'uid': '8f3356df-6d4a-4f5f-880b-e2315bfd29a6',
              'x': array([20000, 20048, 20096, ..., 48656, 48704, 48752]),
              'y': array([1, 1, 1, ..., 1, 1, 1])},
             {'marker': {'color': 'rgba(153, 170, 187, 1)', 'symbol': 'circle'},
              'meta': {'step': 48},
              'mode': 'markers',
              'name': 'Past model evaluations',
              'showlegend': False,
              'type': 'scatter',
              'uid': '34ec5afe-31f5-4426-b61e-4b7f661033d6',
              'x': array([    0,    48,    96, ..., 19824, 19872, 19920]),
              'y': array([1, 1, 1, ..., 1, 1, 1])},
             {'marker': {'symbol': 'circle'},
              'mode': 'markers',
              'name': 'Updates sf_active',
              'type': 'scatter',
              'uid': '76b0f03d-48

### Animation of episode evolution

After generating the static images, check [this gist]() to see how to animate the evolution of the episodes into a gif.

```bash
magick *.png -morph 3 -delay 50 -loop 0 output.gif
```

In [25]:
# Times
(output_path / "time_version").mkdir(exist_ok=True, parents=True)
for opt_step_idx in range(0, (episode_samples-optim_window_samples)//step_samples_opt):
    fig = generate_optim_cycle_viz(
        start = opt_step_idx * sample_time_opt, 
        episode_span = episode_duration,
        optim_window_span = optim_window_size,
        mod_eval_span = sample_time_mod,
        dec_var_updates=dec_var_updates,
        use_times=True,
    )
    fig.write_image(output_path / f"time_version/step_{opt_step_idx:03d}.png", scale=2)
    
# Samples
(output_path / "samples_version").mkdir(exist_ok=True, parents=True)
for opt_step_idx in range(0, (episode_samples-optim_window_samples)//step_samples_opt):
    fig = generate_optim_cycle_viz(
        start = opt_step_idx * step_samples_opt, 
        episode_span = episode_samples,
        optim_window_span = optim_window_samples,
        mod_eval_span = n_evals_mod,
        dec_var_updates=dec_var_updates,
        use_times=False
    )
    fig.write_image(output_path / f"samples_version/step_{opt_step_idx:03d}.png", scale=2)
